In [5]:
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score, precision_score, recall_score, f1_score
import numpy as np
import os
import cv2
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
import sys
sys.path.append("../")
from src.features.lbp_extractor import LBPExtractor

In [6]:
attack_dir=r"../data/Detectedface/ImposterFace"
real_dir=r"../data/Detectedface/ClientFace"
extractor=LBPExtractor()
x=[]
y=[]

In [7]:

count = 0
for root, dirs, files in os.walk(real_dir):
    for filename in files:
        if filename.lower().endswith(('.jpg', '.png', '.jpeg', '.bmp')):
            img_path = os.path.join(root, filename)
            img = cv2.imread(img_path)
            if img is not None:
                try:
                    img_resized = cv2.resize(img, (128, 128))
                    feat = extractor.extract(img_resized)
                    x.append(feat)
                    y.append(1)
                    count += 1
                except Exception as e:
                    print(f"Error processing {filename}: {e}")
print(f"Loaded {count} real images.")

Loaded 5105 real images.


In [8]:
count=0
for root,dirs,files in os.walk(attack_dir):
  for filename in files:
    if filename.lower().endswith(('.jpg','.png','.jpeg','.bmp')):
      img_path=os.path.join(root,filename)
      img=cv2.imread(img_path)
      if img is not None:
        try:
          img_resized=cv2.resize(img,(128,128))
          feat=extractor.extract(img_resized)
          x.append(feat)
          y.append(0)
          count+=1
        except Exception as e:
          print(f"Error processing {filename}: {e}")
print(f"Loaded {count} imposter images.")

Loaded 7509 imposter images.


In [9]:
xtr,xte,ytr,yte=train_test_split(x,y,test_size=0.2,random_state=21)

In [10]:
scaler=StandardScaler()
xtr=scaler.fit_transform(xtr)
xte=scaler.transform(xte)

In [11]:
param_grid={'C':[0.1,1,10,100],'gamma':['scale','auto',0.01,0.001],'kernel':['rbf']}
grid=GridSearchCV(estimator=SVC(class_weight="balanced"),param_grid=param_grid,scoring="f1",cv=3,n_jobs=-1,verbose=1)
grid.fit(xtr,ytr)
model=grid.best_estimator_

Fitting 3 folds for each of 16 candidates, totalling 48 fits


In [12]:
y_preds=model.predict(xte)

In [13]:
from sklearn.metrics import classification_report
print(classification_report(yte,y_preds))

              precision    recall  f1-score   support

           0       0.99      1.00      1.00      1480
           1       1.00      0.99      0.99      1043

    accuracy                           0.99      2523
   macro avg       0.99      0.99      0.99      2523
weighted avg       0.99      0.99      0.99      2523



In [14]:
model=grid.best_estimator_

In [15]:
ypreds=model.predict(xte)

In [16]:
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report,f1_score,precision_score
print(accuracy_score(yte,ypreds))
print(confusion_matrix(yte,ypreds))
print(classification_report(yte,ypreds))
print(f1_score(yte,ypreds))
print(precision_score(yte,ypreds))

0.9944510503369005
[[1476    4]
 [  10 1033]]
              precision    recall  f1-score   support

           0       0.99      1.00      1.00      1480
           1       1.00      0.99      0.99      1043

    accuracy                           0.99      2523
   macro avg       0.99      0.99      0.99      2523
weighted avg       0.99      0.99      0.99      2523

0.9932692307692308
0.996142719382835
